[Open In Colab](https://colab.research.google.com/github/joelbenjohn/pyCUFSM/blob/master/Jupyter_Notebooks/pyCUFSM_load_mat.ipynb)

# Clone Github

In [ ]:
!rm -rf /content/pyCUFSM
!git clone https://github.com/ClearCalcs/pyCUFSM
import sys

sys.path.insert(1, "/content/pyCUFSM")

# Load .mat File

ModuleNotFoundError: No module named 'google'

# Read variables from .mat file

In [73]:
from scipy import io
import pycufsm.helpers as helpers

file = io.loadmat("a.mat")
cufsm_input = helpers.load_cufsm_matsm_mat(file)

<class 'dict'>


UnicodeDecodeError: 'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)

# Preprocessing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycufsm.fsm import strip
from pycufsm.preprocess import stress_gen
from pycufsm.preprocess import yield_mp
from pycufsm.preprocess import doubler
import pycufsm.plotters as plotters
from pycufsm.cutwp import prop2 as cutwp_prop2

nodes = cufsm_input["nodes"]
elements = cufsm_input["elements"]
props = cufsm_input["props"]
springs = cufsm_input["springs"]
constraints = cufsm_input["constraints"]
GBT_con = cufsm_input["GBTcon"]
lengths = cufsm_input["lengths"]
sect_props = cutwp_prop2(nodes[:, 1:3], elements[:, 1:4])
# Values here correspond to signature curve basis and orthogonal based upon geometry
GBT_con = {
    "glob": [0],
    "dist": [0],
    "local": [0],
    "other": [0],
    "o_space": 1,
    "couple": 1,
    "orth": 2,
    "norm": 0,
}
# Simply-supported boundary conditions
B_C = "S-S"

# For signature curve analysis, only a single array of ones makes sense here
m_all = np.ones((len(lengths), 1))

# Solve for 10 eigenvalues
n_eigs = 10

# Flag = {node, elem, mat, stress, stresspic, coord, constraints, springs, origin, propaxis}
flag = np.array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0])
plotters.crossect(nodes, elements, springs, constraints, flag)

# Yield Strength
f_y = 50
restrained = False  # i.e. True do a restrained bending calculation
# Generate the stress points assuming 500 MPa yield and X-axis bending
Fyield = yield_mp(nodes, f_y, sect_props, restrained)
# Generate stress points
nodes_p = stress_gen(
    nodes=nodes,
    forces={
        "P": Fyield["P"] * 0,
        "Mxx": Fyield["Mxx"] * 1,
        "Myy": Fyield["Myy"] * 0,
        "M11": Fyield["M11"] * 0,
        "M22": Fyield["M22"] * 0,
    },
    sect_props=sect_props,
    restrained=restrained,
)

# Analysis

In [ ]:
# Perform the Finite Strip Method analysis
signature, curve, shapes = strip(
    props=props,
    nodes=nodes_p,
    elements=elements,
    lengths=lengths,
    springs=springs,
    constraints=constraints,
    GBT_con=GBT_con,
    B_C=B_C,
    m_all=m_all,
    n_eigs=n_eigs,
    sect_props=sect_props,
)
signature = np.array(signature)
curves = np.array(curve)
shapes = np.array(shapes)
curve = np.zeros(((len(lengths), n_eigs, 2)))
for j in range(len(lengths)):
    for i in range(n_eigs):
        curve[j, i, 0] = lengths[j]
        curve[j, i, 1] = curves[j, i]
fileindex = 1
fileddisplay = [1]
clas = 0
clasopt = 0

# Post Processing
## Displaced Shape

In [ ]:
######POST PROCESSING#########
modeindex = 1
length_index = 20
scale = 5
x = plotters.dispshap(1, nodes, elements, shapes[length_index - 1, :, modeindex - 1], scale, springs, [1], B_C, 1 / 2)

## Signature Curve

In [ ]:
####SIGNATURE CURVE####
###Buckling halfwavelenth plot from a signature analysis
minopt = 1
logopt = 1

# plot limits
xmin = np.min(lengths) * 10 / 11
xmax = np.max(lengths) * 11 / 10
ymin = 0
ymax = np.min([np.max(signature), 3 * np.median(signature)])

# Modes to display
modedisplay = [1]

# Interested point
modeindex = 1
length_index = 40
picpoint = [lengths[length_index - 1], curves[length_index - 1, modeindex - 1]]

# CALL THE CURVE
plotters.thecurve3(
    curve,
    clas,
    fileddisplay,
    minopt,
    logopt,
    clasopt,
    xmin,
    xmax,
    ymin,
    ymax,
    modedisplay,
    fileindex,
    modeindex,
    picpoint,
)